In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [14]:
import tensorflow
import keras

In [15]:
%matplotlib inline

In [16]:
train_y = pd.read_csv(r"C:\Users\hp pc\OneDrive\New folder\train_v2.csv\train_v2.csv")

In [17]:
train_y.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [34]:
import os
import gc
import seaborn as sns
pal = sns.color_palette()

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [35]:
labels = train_y['tags'].apply(lambda x: x.split(' '))

In [36]:
#print(labels)
from collections import Counter, defaultdict
counts = defaultdict(int)
for l in labels:
    for l2 in l:
        counts[l2] += 1

data=[go.Bar(x=list(counts.keys()), y=list(counts.values()))]
layout=dict(height=800, width=800, title='Distribution of training labels')
fig=dict(data=data, layout=layout)
py.iplot(data, filename='train-label-dist')

In [37]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in train_y['tags'].values])))
label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

In [40]:
from tqdm import tqdm
import cv2

In [41]:
x_train=[]
y_train=[]
x_test = []

In [44]:
for f,tags in tqdm(train_y.values,miniters=1000):
    img = cv2.imread(r"C:\Users\hp pc\OneDrive\New folder\train-jpg.tar (1)\train-jpg\{}.jpg".format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]]=1
        
        
    x_train.append(cv2.resize(img,(32,32)))
    y_train.append(targets)
    
    

100%|████████████████████████████████████████████████████████████████████████████| 40479/40479 [23:57<00:00, 28.16it/s]


In [46]:
x_train = np.array(x_train)

In [50]:
#print(x_train[1])
y_train = np.array(y_train)

In [51]:
y_train[1]

array([ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  1.])

In [52]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU

In [60]:
def make_model():
    """
    Define your model architecture here.
    Returns `Sequential` model.
    """
    model = Sequential()

    ### YOUR CODE HERE
    model.add(Conv2D(16,kernel_size = (3,3),padding ="same",input_shape=(32,32,3)))
    model.add(LeakyReLU(0.1))
    model.add(Conv2D(32,kernel_size=(3,3),padding="same"))
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2,2),padding="same"))
    model.add(Dropout(0.25))
    model.add(Conv2D(32,kernel_size=(3,3),padding="same"))
    model.add(LeakyReLU(0.1))
    model.add(Conv2D(64,kernel_size=(3,3),padding="same"))
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2,2),padding="same"))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))
    model.add(Dense(17))
    model.add(Activation("softmax"))
    
    
    
    return model

In [61]:
print(keras.__version__)
from keras import backend as K

2.0.6


In [62]:
K.clear_session()  # clear default graph
model = make_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 32)        9248      
__________

In [63]:
INIT_LR = 5e-3  # initial learning rate
BATCH_SIZE = 32
EPOCHS = 10

K.clear_session()  # clear default graph
# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss="binary_crossentropy",  # we train 10-way classification
    optimizer=keras.optimizers.adamax(lr=INIT_LR),  # for SGD
    metrics=['accuracy']  # report accuracy during training
)

# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback for printing of actual learning rate used by optimizer
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

In [64]:
x_train,y_train,x_valid,y_valid = x_train[:3400],y_train[:3400],x_train[3400:],y_train[3400:]

In [ ]:
model.fit(
    x_train, y_train,  # prepared data
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), LrHistory()],
    validation_data=(x_valid,y_valid),
    shuffle=True,
)